# 20151230-predict-household-income-from-census

Related post:  
https://stharrold.github.io/20151230-predict-household-income-from-census.html

Purpose: Predict total annual household income.

## Initialization

### Imports

In [1]:
cd ~

/home/samuel_harrold


In [3]:
# Import standard packages.
import collections
import os
import pdb # TEST: Comment out pdb after testing.
import sys
# Import installed packages.
import matplotlib.pyplot as plt
import pandas as pd
# Import local packages.
sys.path.insert(
    0,
    os.path.join(os.path.curdir, 'stharrold.github.io/content/static/dsdemos'))
# TEST: Comment out autoreload after testing.
%reload_ext autoreload
%autoreload 2
import dsdemos as dsd
# IPython magic.
%matplotlib inline

## Globals

In [4]:
path_static = os.path.join(os.path.expanduser(r'~'), r'stharrold.github.io/content/static')
basename = r'20151230-predict-household-income-from-census'
path_disk = os.path.abspath(r'/mnt/disk-20151227t211000z/')
path_acs = os.path.join(path_disk, r'www2-census-gov/programs-surveys/acs/')
path_csv = os.path.join(path_acs, r'data/pums/2013/5-Year/ss13hdc.csv') # 'hdc' = 'housing DC'
path_ddict = os.path.join(path_acs, r'tech_docs/pums/data_dict/PUMSDataDict13.txt')

## Extract-transform-load

**TODO:**
* Just use pandas. Acknowledge dask.

In [5]:
%%time
with open(path_csv) as fobj:
    nlines = sum(1 for _ in fobj)
print("{path}:".format(path=path_csv))
print("size (MB) = {size:.1f}".format(size=os.path.getsize(path_csv)/1e6))
print("num lines = {nlines}".format(nlines=nlines))
df = pd.read_csv(path_csv)
print("df RAM usage (MB) = {mem:.1f}".format(mem=df.memory_usage().sum()/1e6))

/mnt/disk-20151227t211000z/www2-census-gov/programs-surveys/acs/data/pums/2013/5-Year/ss13hdc.csv:
size (MB) = 13.5
num lines = 17501
df RAM usage (MB) = 28.7
CPU times: user 412 ms, sys: 56 ms, total: 468 ms
Wall time: 466 ms


In [6]:
percentiles = [0.1587, 0.5000, 0.8413] # +1 std. dev., mean/median, -1 std. dev. for normal dist.
df.describe(percentiles=percentiles, include='all')

,insp,RT,SERIALNO,DIVISION,PUMA00,PUMA10,REGION,ST,ADJHSG,ADJINC,...,WGTP71,WGTP72,WGTP73,WGTP74,WGTP75,WGTP76,WGTP77,WGTP78,WGTP79,WGTP80
count,6561.000000,17500,1.750000e+04,17500,17500.000000,17500.000000,17500,17500,17500.000000,17500.000000,...,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000,17500.000000
unique,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,17500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,999.282731,NaN,2.011068e+12,5,56.427371,37.764171,3,11,1039364.231657,1048478.770229,...,17.050857,17.043486,17.050000,17.049029,17.048000,17.051543,17.053200,17.047029,17.046971,17.051486
std,1085.174484,NaN,1.401911e+09,0,55.291036,55.358495,0,0,31877.254257,29598.269890,...,17.593886,17.740566,17.534604,17.555515,17.558942,17.574232,17.623017,17.802284,17.267472,17.710924
min,0.000000,NaN,2.009000e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
15.9%,200.000000,NaN,2.009001e+12,5,-9.000000,-9.000000,3,11,1000000.000000,1007549.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
50%,790.000000,NaN,2.011001e+12,5,101.000000,-9.000000,3,11,1035725.000000,1054614.000000,...,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
84.1%,1500.000000,NaN,2.013000e+12,5,104.000000,104.000000,3,11,1086032.000000,1085467.000000,...,30.000000,30.000000,30.000000,31.000000,30.000000,30.000000,30.000000,30.000000,31.000000,30.000000


In [159]:
ddict = dsd.census.parse_pumsdatadict13(path=path_ddict)

In [160]:
ddict

OrderedDict([('title', '2013 ACS PUMS DATA DICTIONARY'),
             ('date', 'August 7, 2015'),
             ('record_types',
              OrderedDict([('HOUSING RECORD',
                            OrderedDict([('RT',
                                          OrderedDict([('length', '1'),
                                                       ('description',
                                                        'Record Type'),
                                                       ('var_codes',
                                                        OrderedDict([('H',
                                                                      'Housing Record or Group Quarters Unit')]))])),
                                         ('SERIALNO',
                                          OrderedDict([('length', '7'),
                                                       ('description',
                                                        'Housing unit/GQ person serial number'),
       

## Export ipynb to html

In [158]:
!date --rfc-3339='seconds'

2016-01-01 05:40:17+00:00


In [ ]:
path_ipynb = os.path.join(path_static, basename, basename+'.ipynb')
for template in ['basic', 'full']:
    path_html = os.path.splitext(path_ipynb)[0]+'-'+template+'.html'
    cmd = ['jupyter', 'nbconvert', '--to', 'html', '--template', template, path_ipynb, '--output', path_html]
    print(' '.join(cmd))
    subprocess.run(args=cmd, check=True)